In [1]:
from __future__ import division
from pyomo.environ import *
import numpy as np
import pandas as pd
import time
from functools import reduce
import itertools
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)
from pprint import pprint

In [2]:
product_data = pd.read_csv('data/AnnualOptimizerInputData.csv')
last_quarter_product_base = pd.read_csv('data/LastQuarterData.csv')
coefficient_matrix = pd.read_csv('data/coefficient_matrix.csv')

In [3]:
products = product_data.shape[0]
idx = range(products)
weeks = 52
tot = products*weeks

In [4]:
coef = []
coefficient_matrix_dr = coefficient_matrix.drop(columns=['Modelled_Product','wk'])#'Product1_Retailer_EDLP_Product6_Retailer_EDLP'])

for i in range(products):
    coef.append(coefficient_matrix_dr.iloc[i::6, :].values.tolist())

In [5]:
base = last_quarter_product_base['Base Price']
base_price = [[x]*weeks for x in base]

In [6]:
target_trade_spend = product_data.Product_Total_Spend.tolist()
total_target_trade_spend_final = sum(target_trade_spend)
target_edlp_spend = product_data.Product_EDLP_Spend.tolist()
target_tpr_spend = product_data.Product_TPR_Spend.tolist()
target_tpr_events = product_data.TPR_Events.tolist()

target_trade_spend_lower_bd = [x * 0.9 for x in target_trade_spend]
target_trade_spend_upper_bd = [x * 1.1 for x in target_trade_spend]

target_edlp_spend_lower_bd = [x * 0.8 for x in target_edlp_spend]
target_edlp_spend_upper_bd = [x * 1.2 for x in target_edlp_spend]

target_tpr_spend_lower_bd = [x * 0.8 for x in target_tpr_spend]
target_tpr_spend_upper_bd = [x * 1.2 for x in target_tpr_spend]

target_tpr_events_lb = product_data.TPR_Events_Min.tolist()
target_tpr_events_ub = product_data.TPR_Events_Max.tolist()

In [7]:
import re
def get_info_from_results(results, info_string):
    i = str(results).lower().find(info_string.lower()) + len(info_string)
    value = ''
    while str(results)[i] != '\n':
        value = value + str(results)[i]
        i += 1
    return float(value)
def get_obj_frm_res(results, info_string):
#     print(results)
    results = results.lower()
    info_string = info_string.lower()
    z = re.search(info_string, results).start()
    
    return(float(results[z+108:z+125]))

In [16]:
def calculate_interaction(prd_no, inter_prd, edlp, flag,i):

    p1 = base_price[prd_no][i] * (1-flag[prd_no,i]) + base_price[prd_no][i]* edlp[prd_no,i] * (flag[prd_no,i])
    p2 = base_price[inter_prd][i] * (1-flag[inter_prd,i]) + base_price[inter_prd][i]* edlp[inter_prd,i] * (flag[inter_prd,i])
    
    return log(p1)*log(p2)

def cal_sales_prior(edlp,tpr,flag,coef,j,factors):
    unit_sales = []
    for i in  range(weeks):
        edlp_sum = 0
        tpr_sum = 0
        total = 0
        cc= 0
        for p in range(products):
            if p==j:
                z = 0
                edlp_sum+= (log( base_price[p][i] * edlp[p,i])*coef[i][cc+1])* z
                tpr_sum+= (log(base_price[p][i])*coef[i][cc+1]+ 100*tpr[p,i]*z)*(1- z)
            else:
                edlp_sum+= (log( base_price[p][i] * edlp[p,i])*coef[i][cc+1])* flag[p,i]
                tpr_sum+= (log(base_price[p][i])*coef[i][cc+1]+ 100*tpr[p,i]*coef[i][cc])*(1- flag[p,i])
                
            cc+=2
            
        try:
            interaction = calculate_interaction(j, factors[j], edlp,flag,i)
        except:
            interaction = 1
            
        unit_sales.append(exp(edlp_sum+ tpr_sum + coef[i][-2] + interaction*coef[i][-1]))                 
    return unit_sales
    
def cal_sales(edlp,tpr,flag,coef,j,factors):
    unit_sales = []
    for i in  range(weeks):
        edlp_sum = 0
        tpr_sum = 0
        cc= 0
        for p in range(products):
            edlp_sum+= (log( base_price[p][i] * edlp[p,i])*coef[i][cc+1])* flag[p,i]
            tpr_sum+= (log(base_price[p][i])*coef[i][cc+1]+ 100*tpr[p,i]*coef[i][cc])*(1- flag[p,i])
            cc+=2
        try:
            interaction = calculate_interaction(j, factors[j], edlp,flag,i)
        except:
            interaction = 1
        unit_sales.append(exp(edlp_sum+ tpr_sum + coef[i][-2] + interaction*coef[i][-1]))            
    return unit_sales

def calc_price(edlp,tpr,flag,j):
    price = []
    for i in range(weeks):
        price.append((base_price[j][i]*(1-tpr[j,i])*(1-flag[j,i]))  +  (base_price[j][i]*edlp[j,i]*flag[j,i]))
    return price

def calc_dollar_sales(edlp,tpr,flag,coef,interaction_factors, prior=False):
    dollar_sales=[]
    price = []
    unit_sales = []
    
    for j in range(products):
        if not prior:
            unit_sales_single= cal_sales(edlp,tpr,flag,coef[j],j,interaction_factors)
            price_single = calc_price(edlp,tpr,flag,j)
        else:
            unit_sales_single = cal_sales_prior(edlp,tpr,flag,coef[j],j,interaction_factors)
            price_single = base_price[j]

        dollar_sales_single = [unit_sales_single[i]*price_single[i] for i in range(weeks)]
        dollar_sales.append(dollar_sales_single)
        price.append(price_single)
        unit_sales.append(unit_sales_single)
    
    return dollar_sales, unit_sales, price

def calc_total_trade_spent(edlp,tpr,flag,coef, unit_sales, price):
    total_trade_spent = []
    for j in range(products):
        trade_spent = [base_price[j][i]-price[j][i] for i in range(weeks)]
        total_trade_spent_partial = [trade_spent[i]*unit_sales[j][i] for i in range(weeks)]
        total_trade_spent.append(total_trade_spent_partial)
        
    return total_trade_spent

def calc_edlp_trade_spent(edlp,tpr,flag,coef,total_trade_spent):
    edlp_trade_spent = []
#     flag = [flag_p1,flag_p2]
    for j in range(products):
        edlp_trade_spent_partial = [total_trade_spent[j][i]*flag[j,i] for i in range(weeks)]
        edlp_trade_spent.append(edlp_trade_spent_partial)
    return edlp_trade_spent


def calculate_lift(dollar_sales, prior_prom_sales):
    lift = []
    for i in range(products):
        for j in range(weeks):
            lift.append(dollar_sales[i][j] - prior_prom_sales[i][j])
    return lift

def calculate_total_tpr_events(flag):
    tpr_events = []
    for i in range(products):
        temp_sum = 0
        for j in range(weeks):
            temp_sum+=flag[i,j]
        tpr_events.append(weeks-temp_sum)
    return tpr_events

def calculate_tpr_trade_spend(total_trade_spent, edlp_trade_spent):
    tpr_spend =[]
    for i in range(products):
        tpr_spend.append([a-b for a,b in zip(total_trade_spent[i], edlp_trade_spent[i])])
    return tpr_spend

In [17]:
def create_model(init, interaction_prds):
    print("Init value= ",init)
    def initial(model,i,j):
        return init[i]

    model = ConcreteModel(name='Spend_Optim')
    model.weeks = Param(initialize=weeks, domain=PositiveIntegers)
    model.ppgs = Param(initialize=products, domain=PositiveIntegers)
    model.wk_index = RangeSet(0, model.weeks - 1)
    model.ppg_index = RangeSet(0, model.ppgs - 1)

    model.edlp = Var(model.ppg_index, model.wk_index, initialize=0.95, bounds=(0.95, 1), domain=NonNegativeReals)
    model.tpr = Var(model.ppg_index, model.wk_index, initialize=0.05, bounds=(0.05, 0.50), domain=NonNegativeReals)
    model.flag = Var(model.ppg_index, model.wk_index, initialize=0, domain=Binary)
    
    prior_prom_sales, prior_units, prior_price = calc_dollar_sales(model.edlp,model.tpr,model.flag,coef, interaction_prds,True)
    dollar_sales, units, price = calc_dollar_sales(model.edlp,model.tpr,model.flag,coef, interaction_prds)
    
    total_trade_spent = calc_total_trade_spent(model.edlp,model.tpr,model.flag,coef,units,price)
    edlp_trade_spent = calc_edlp_trade_spent(model.edlp,model.tpr,model.flag,coef,total_trade_spent)
    tpr_trade_spend = calculate_tpr_trade_spend(total_trade_spent, edlp_trade_spent)
    
#     lift = calculate_lift(dollar_sales, prior_prom_sales)
    sum_of_total_trade_spent = [sum(total_trade_spent[i]) for i in range(products)]
    tpr_events = calculate_total_tpr_events(model.flag)
    
    total_obj = [sum(dollar_sales[i]) for i in range(products)]
    
    model.obj = Objective(expr= sum(total_obj) ,sense= maximize)
    
    model.total_spend_constraint = Constraint(expr = sum(sum_of_total_trade_spent) == total_target_trade_spend_final)
#     model.dummy = Constraint(expr = Constraint.Feasible)
    for i in range(products):
        name_1 = 'constraint_bound_total_product_' + str(i)
        name_3 = 'constraint_bound_edlp_product_' + str(i)
        name_4 = 'constraint_bound_tpr_product_' + str(i)
        name_5 = 'constraint_bound_tpr_events_product_' + str(i)
        
#         model.add_component(name_1, Constraint(expr = target_trade_spend_upper_bd[i] >= sum(total_trade_spent[i]) >= target_trade_spend_lower_bd[i]))
#         model.add_component(name_3, Constraint(expr = target_edlp_spend_upper_bd[i] >= sum(edlp_trade_spent[i]) >= target_edlp_spend_lower_bd[i]))
#         model.add_component(name_4, Constraint(expr = target_tpr_spend_upper_bd[i] >= sum(tpr_trade_spend[i]) >= target_tpr_spend_lower_bd[i]))
#         model.add_component(name_5, Constraint(expr = target_tpr_events_ub[i] >= tpr_events[i] >= target_tpr_events_lb[i]))
    return model

In [18]:
def call_solver(init,interaction_prds, name='bonmin'):
    assert type(init) == list
    model = create_model(init,interaction_prds)
    opt_1 = SolverFactory(name)
    results_1 = opt_1.solve(model)
    t = get_info_from_results(results_1, 'Time: ')
    print('Time taken is: ', t)
    model.display('hola.txt')
    file =  open('hola.txt', 'r')
    data = file.read()
    o = get_obj_frm_res(data, 'Objectives')
    file.close()
    print('Objective is: ', o)
    print('-------------------------------------------------')
    return o,t,model

In [19]:
time_lst = []
obj_lst = []
time_lst_1 = []

interaction_factors = {}
interaction_factors[0] = 5
interaction_factors[5] = 0


start_time = time.time()
t_lst, o_lst,model = call_solver([],interaction_factors, 'bonmin')
end_time = time.time()
time_lst.append(t_lst)
obj_lst.append(o_lst)
time_lst_1.append(end_time - start_time)
model.display()
#     break
# time_lst, obj_lst
# 5566396.813079901

Init value=  []
  Signal handler called from  /home/tiger0492/nilagnik/mini_installation/miniconda/envs/optim_env/lib/python3.7/subprocess.py _try_wait 1559
  Waiting...
  Signaled process 68 with signal 2
ERROR: Solver (asl) returned non-zero return code (-1)
ERROR: Solver log: Bonmin 1.8.6 using Cbc 2.9.9 and Ipopt 3.12.8 bonmin:

    **************************************************************************
        **** This program contains Ipopt, a library for large-scale nonlinear
        optimization.
     Ipopt is released as open source code under the Eclipse Public License
     (EPL).
             For more information visit http://projects.coin-or.org/Ipopt
    **************************************************************************
        ****

    NLP0012I
                  Num      Status      Obj             It       time
                  Location
    NLP0014I             1         OPT -9338536.9      169 1.09375 NLP0012I
                  Num      Status      Obj    

ApplicationError: Solver (asl) did not exit normally

In [10]:
def cal_sales_prior(edlp,tpr,flag,coef,j):
    unit_sales = []
    for i in  range(weeks):
        edlp_sum = 0
        tpr_sum = 0
        total = 0
        cc= 0
        for p in range(products):
            if p==j:
                z = 0
                edlp_sum+= (log( base_price[p][i] * edlp[p,i])*coef[i][cc])* z
                tpr_sum+= (log(base_price[p][i])*coef[i][cc]+ tpr[p,i]*z)*(1- z)
            else:
                edlp_sum+= (log( base_price[p][i] * edlp[p,i])*coef[i][cc])* flag[p,i]
                tpr_sum+= (log(base_price[p][i])*coef[i][cc]+ tpr[p,i]*coef[i][cc+1])*(1- flag[p,i])
                
            cc+=2
            
        unit_sales.append(exp(edlp_sum+ tpr_sum +coef[i][-1]))                 
    return unit_sales

def cal_sales(edlp,tpr,flag,coef,j):
    unit_sales = []
    for i in  range(weeks):
        edlp_sum = 0
        tpr_sum = 0
        cc= 0
        for p in range(products):
            edlp_sum+= (log( base_price[p][i] * edlp[p,i])*coef[i][cc])* flag[p,i]
            tpr_sum+= (log(base_price[p][i])*coef[i][cc]+ tpr[p,i]*coef[i][cc+1])*(1- flag[p,i])
            cc+=2
        unit_sales.append(exp(edlp_sum+ tpr_sum +coef[i][-1]))                 
   
    return unit_sales

def calc_price(edlp,tpr,flag,j):
    price = []
    for i in range(weeks):
        price.append((base_price[j][i]*(1-tpr[j,i])*(1-flag[j,i]))  +  (base_price[j][i]*edlp[j,i]*flag[j,i]))
                   
    return price
                   
def calc_dollar_sales(edlp,tpr,flag,coef, prior=False):
    dollar_sales=[]
    for j in range(products):
        if not prior:
            unit_sales= cal_sales(edlp,tpr,flag,coef[j],j)
            price = calc_price(edlp,tpr,flag,j)
        else:
            unit_sales = cal_sales_prior(edlp,tpr,flag,coef[j],j)
            price = base_price[j]

        dollar_sales_single = [unit_sales[i]*price[i] for i in range(weeks)]
        dollar_sales.append(dollar_sales_single)
    
    return dollar_sales


def calc_total_trade_spent(edlp,tpr,flag,coef):
    total_trade_spent = []
    for j in range(products):
        unit_sales= cal_sales(edlp,tpr,flag,coef[j],j)
        price = calc_price(edlp,tpr,flag,j)

        trade_spent = [base_price[j][i]-price[i] for i in range(weeks)]
        total_trade_spent_partial = [trade_spent[i]*unit_sales[i] for i in range(weeks)]
        total_trade_spent.append(total_trade_spent_partial)
        
    return total_trade_spent

def calc_edlp_trade_spent(edlp,tpr,flag,coef):
    edlp_trade_spent = []
#     flag = [flag_p1,flag_p2]
    for j in range(products):
        unit_sales= cal_sales(edlp,tpr,flag,coef[j],j)
        price = calc_price(edlp,tpr,flag,j)

        trade_spent = [base_price[j][i]-price[i] for i in range(weeks)]
        total_trade_spent_partial = [trade_spent[i]*unit_sales[i] for i in range(weeks)]

        edlp_trade_spent_partial = [total_trade_spent_partial[i]*flag[j,i] for i in range(weeks)]
        edlp_trade_spent.append(edlp_trade_spent_partial)
    return edlp_trade_spent

def calculate_tpr_trade_spend(total_trade_spent, edlp_trade_spent):
    tpr_spend =[]
    for i in range(products):
        tpr_spend.append([a-b for a,b in zip(total_trade_spent[i], edlp_trade_spent[i])])
    return tpr_spend

def calculate_lift(dollar_sales, prior_prom_sales):
    lift = []
    for i in range(products):
        for j in range(weeks):
            lift.append(dollar_sales[i][j] - prior_prom_sales[i][j])
    return lift

def calculate_total_tpr_events(flag):
    tpr_events = []
    for i in range(products):
        temp_sum = 0
        for j in range(weeks):
            temp_sum+=flag[i,j]
        tpr_events.append(weeks-temp_sum)
    return tpr_events